In [1]:
gpu_info = !nvidia-smi
gpu_info = "\n".join(gpu_info)
if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU")
else:
    print(gpu_info)

Fri Oct  6 17:39:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
print("Your runtime has {:.1f} gigabytes of available RAM\n".format(ram_gb))

if ram_gb < 20:
    print("Not using a high-RAM runtime")
else:
    print("You are using a high-RAM runtime!")

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install wandb -qU
!pip install -U tensorflow
!pip install python-dotenv

In [4]:
from google.colab import drive

# Mount the Google Drive to access the files
drive.mount("/content/gdrive/")
work_directory = "/content/gdrive/MyDrive/wsi_code"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [5]:
import os
import sys
from datetime import datetime
import numpy as np
import logging

# Add the path to your project root directory
if work_directory not in sys.path:
    sys.path.append(work_directory)

# my utility functions
from utils.general import create_directory
from utils.dataloader import select_case_data

# TensorFlow and Keras imports
import tensorflow as tf
from keras.layers import (
    Dense,
    GlobalAveragePooling2D,
)

from keras.optimizers import Adam
from keras.callbacks import (
    ReduceLROnPlateau,
    TensorBoard,
)
import wandb

# load env variables
from dotenv import load_dotenv

load_dotenv(os.path.join(work_directory, ".env"))

True

In [6]:
# Define data directories
DATASETS_PATH = os.path.join(work_directory, "datasets")
PROCESSED_PATH = os.path.join(DATASETS_PATH, "processed")
hdf5_file = os.path.join(PROCESSED_PATH, "patchs_384_40k.hdf5")
run_dir = os.path.join(work_directory, "runs", "40k")

# Create directories with datetime
model_dir = os.path.join(run_dir, "efficientnetb0")

# Create the directories
create_directory(model_dir)

# Get the current datetime
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

log_path = os.path.join(model_dir, f"{current_datetime}.log")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    filename=log_path,  # Specify the file name and path
    filemode="w",  # 'w' for write mode, use 'a' to append to an existing file
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

Directory already exists: /content/gdrive/MyDrive/wsi_code/runs/40k/efficientnetb0


In [7]:
# Load and preprocess the data
logger.info("Loading and preprocessing data...")
validation_images, validation_labels, train_images, train_labels = select_case_data(
    hdf5_file, selected_cases=[4]
)

# Define a normalization layer
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

# One-hot encode the labels
num_classes = np.unique(train_labels).shape[
    0
]  # Replace with the actual number of classes
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
validation_labels = tf.keras.utils.to_categorical(validation_labels, num_classes)


def preprocess_data(images, labels):
    images = normalization_layer(images)
    return images, labels


def create_and_preprocess_dataset(
    images, labels, batch_size, augment=False, shuffle_buffer_size=1000
):
    # Create a dataset from the input images and labels
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))

    # Shuffle the dataset for randomness
    dataset = dataset.shuffle(shuffle_buffer_size)

    if augment:
        # Apply data augmentation within the dataset pipeline
        dataset = dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
        dataset = dataset.map(lambda x, y: (tf.image.random_flip_up_down(x), y))
        dataset = dataset.map(
            lambda x, y: (tf.image.random_brightness(x, max_delta=0.05), y)
        )
        dataset = dataset.map(
            lambda x, y: (tf.image.random_contrast(x, lower=0.9, upper=1.1), y)
        )

    # Normalize the images
    dataset = dataset.map(preprocess_data)

    # Batch the dataset
    dataset = dataset.batch(batch_size)

    # Prefetch for efficient loading during training
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset


# Create TensorFlow datasets and apply normalization
logger.info("Creating datasets and applying normalization...")

batch_size = 16  # You can adjust this based on your available memory
num_parallel_calls = tf.data.AUTOTUNE

normalized_validation_ds = create_and_preprocess_dataset(
    validation_images, validation_labels, batch_size=batch_size
)
normalized_train_ds = create_and_preprocess_dataset(
    train_images, train_labels, augment=False, batch_size=batch_size
)

# Delete unused variables to free up memory
del validation_images, validation_labels, train_images, train_labels

logger.info("Data loading and preprocessing complete.")

In [8]:
efficientnet = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    classifier_activation="softmax",
    input_shape=(384, 384, 3),
)

efficientnet.trainable = False

x = GlobalAveragePooling2D()(efficientnet.output)
x = Dense(256, activation="relu")(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(efficientnet.inputs, outputs, name="EfficientNetV2S")

model.compile(
    optimizer=Adam(learning_rate=5e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)

In [9]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="loss", patience=10, restore_best_weights=True
)

# Define ReduceLROnPlateau callback
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor="loss", factor=0.5, patience=2, min_lr=1e-7
)

# Define TensorBoard callback
tensorboard_callback = TensorBoard(
    log_dir=model_dir,
    histogram_freq=1,  # Enable histogram computation
    write_graph=True,  # Write model graph to file
    write_images=True,  # Write model weights to file
    update_freq="epoch",
)

In [10]:
wandb_api_key = os.getenv("WANDB_API_KEY")

if wandb_api_key:
    wandb.login(key=wandb_api_key)
else:
    print("WANDB_API_KEY not found in the .env file.")

# Before wandb.init, call wandb.tensorboard.patch
wandb.tensorboard.patch(
    root_logdir=model_dir
)  # Replace model_dir with your log directory
wandb.init(
    project="wsi-classification-40k",
    sync_tensorboard=True,
    entity="hacettepe-cerrahpasa-sts",
    notes="efficientnet_cross_4_final",
    tags=["efficientnet", "v2S", "cross_4", "final"],
)
# Initialize wandb callback
wandb_callback = wandb.keras.WandbCallback()

wandb: Currently logged in as: aemreusta (hacettepe-cerrahpasa-sts). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [11]:
# Train the model with callbacks
history = model.fit(
    normalized_train_ds,
    validation_data=normalized_validation_ds,
    epochs=100,
    workers=-1,
    use_multiprocessing=True,
    callbacks=[
        wandb_callback,
        early_stopping,
        reduce_lr_on_plateau,
        tensorboard_callback,
    ],
)

Epoch 1/100
   5/2000 [..............................] - ETA: 1:35 - loss: 1.1610 - categorical_accuracy: 0.9375 - precision: 0.0000e+00 - recall: 0.0000e+00

1999/2000 [============================>.] - ETA: 0s - loss: 0.2229 - categorical_accuracy: 0.9334 - precision: 0.9356 - recall: 0.9228

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231006_174505-n2wa4hje/files/model-best)... Done. 0.3s


2000/2000 [==============================] - 203s 92ms/step - loss: 0.2227 - categorical_accuracy: 0.9335 - precision: 0.9356 - recall: 0.9229 - val_loss: 6.7976 - val_categorical_accuracy: 0.2500 - val_precision: 0.2500 - val_recall: 0.2500 - lr: 5.0000e-05
Epoch 2/100
1999/2000 [============================>.] - ETA: 0s - loss: 0.2679 - categorical_accuracy: 0.9270 - precision: 0.9294 - recall: 0.9235

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Network error resolved after 0:00:11.496002, resuming normal operation.
wandb: Adding directory to artifact (/content/wandb/run-20231006_174505-n2wa4hje/files/model-best)... Done. 0.3s


...

2000/2000 [==============================] - 192s 96ms/step - loss: 0.6528 - categorical_accuracy: 0.7807 - precision: 0.8104 - recall: 0.7295 - val_loss: 2.7090 - val_categorical_accuracy: 0.2499 - val_precision: 0.2500 - val_recall: 0.2498 - lr: 6.2500e-06
Epoch 10/100
2000/2000 [==============================] - ETA: 0s - loss: 1.0372 - categorical_accuracy: 0.5933 - precision: 0.6542 - recall: 0.4531

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231006_174505-n2wa4hje/files/model-best)... Done. 0.3s


2000/2000 [==============================] - 192s 96ms/step - loss: 1.0372 - categorical_accuracy: 0.5933 - precision: 0.6542 - recall: 0.4531 - val_loss: 1.8623 - val_categorical_accuracy: 0.2529 - val_precision: 0.2623 - val_recall: 0.2499 - lr: 3.1250e-06
Epoch 11/100
1999/2000 [============================>.] - ETA: 0s - loss: 0.9587 - categorical_accuracy: 0.6245 - precision: 0.7167 - recall: 0.4699

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231006_174505-n2wa4hje/files/model-best)... Done. 0.3s


2000/2000 [==============================] - 193s 97ms/step - loss: 0.9584 - categorical_accuracy: 0.6247 - precision: 0.7169 - recall: 0.4701 - val_loss: 1.8598 - val_categorical_accuracy: 0.2530 - val_precision: 0.2629 - val_recall: 0.2499 - lr: 3.1250e-06


In [12]:
wandb.finish()

categorical_accuracy,█████▇▇▅▅▁▂
epoch,▁▂▂▃▄▅▅▆▇▇█
global_step,▁▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
loss,▁▁▁▂▂▃▃▅▅█▇
precision,█████▇▇▅▅▁▃
recall,█████▇▇▅▅▁▁
train/epoch_categorical_accuracy,█████▇▇▅▅▁▂
train/epoch_loss,▁▁▁▂▂▃▃▅▅█▇
train/epoch_lr,███▄▄▂▂▁▁▁▁
train/epoch_precision,█████▇▇▅▅▁▃
train/epoch_recall,█████▇▇▅▅▁▁


In [13]:
# runtime.unassign()